In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Finding possible interactors of LIN28A which might cause peri-ER localization

The original article suggested strong evidences such that:
- Integral membrane protein coding mRNAs are highly enriched by LIN28A (and also, their translation highly downregulated)
- Specifically, cotranslationally ER translocated mRNAs are highly enriched by LIN28A
- LIN28A is localized to peri-ER area

But since motif score analysis done by HMM didn't show differences between ER-associated mRNAs and non-ER-associated mRNAs, we still don't know why LIN28A is localized to peri-ER area. So I will start from following hypotheses:
1. LIN28A interacts with ER surface protein/lipid
2. LIN28A interacts with RBP or adapter protein s.t. cotranslationally bounds to ER-associated mRNA.
3. LIN28A directly recognizes ER-associated mRNA sequence/structure/modification context.

Performed some pilot research, but found no evidences supporting hypothesis 1 and 2. Using other CLIP-seq or Ribosome profiling (a.k.a. Ribosome footprinting) data to find other novel recognition site of LIN28A would be better.

In [ ]:
# So let's start from here...

# New research plan: calculating codon occupancy with ribosome profiling data

1. Get P-site offset w.r.t read length
2. Calculate P-site occupancy over exons
3. Calculate and nomalize codon occupancy

In [ ]:
# Manually extract exons with start codon
!grep $'\tstart_codon\t.*\t[+-]\t.*transcript_support_level "1"' ../binfo1-datapack1/gencode.gtf | \
    sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-start.gtf

In [ ]:
!grep $'\texon\t.*\t[+-]\t.*transcript_support_level "1"' ../binfo1-datapack1/gencode.gtf | \
    sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-exon.gtf

In [ ]:
!bedtools intersect -a gencode-start.gtf -b gencode-exon.gtf -wa -wb | \
 awk -F'	' -v OFS='	' '$9 == $18 { print $10, $13-1, $14, $18, $4-1, $16; }' | \
 sort -k1,1 -k2,3n -k4,4 > gencode-exons-containing-startcodon.bed
!head gencode-exons-containing-startcodon.bed; tail gencode-exons-containing-startcodon.bed

In [ ]:
# Now remove duplicates
%cd ../binfo1-datapack1
!stringtie RNA-siLuc.bam -G gencode.gtf -o stringtie-RNA-siLuc.gtf -p 10
!awk -F'\t' '$3 == "transcript"' stringtie-RNA-siLuc.gtf > stringtie-RNA-siLuc-transcript.gtf

In [2]:
%cd ../binfo1-datapack1/

/qbio/mahoon2/bioinfo1/binfo1-datapack1


In [3]:
df = pd.read_csv('stringtie-RNA-siLuc-transcript.gtf', sep='\t', header=None)
df.columns = ['chromosome', 'program', 'type', 'start', 'end', 'unnamed1', 'strand', 'unnamed2', 'info']

In [5]:
df['TPM'] = df['info'].str.extract(r'TPM "([^"]+)"').astype(float)
df['transcript_id'] = df['info'].str.extract(r'reference_id "([^"]+)"')
df.head()

,chromosome,program,type,start,end,unnamed1,strand,unnamed2,info,TPM,transcript_id
0,chr1,StringTie,transcript,3822233,3824583,1000,+,.,"gene_id ""STRG.1""; transcript_id ""STRG.1.1""; re...",0.035499,ENSMUST00000194454.2
1,chr1,StringTie,transcript,4599240,4599346,1000,+,.,"gene_id ""STRG.2""; transcript_id ""STRG.2.1""; re...",0.164673,ENSMUST00000240255.1
2,chr1,StringTie,transcript,4680694,4681629,1000,+,.,"gene_id ""STRG.3""; transcript_id ""STRG.3.1""; re...",1.015851,ENSMUST00000192738.2
3,chr1,StringTie,transcript,4758157,4759626,1000,-,.,"gene_id ""STRG.4""; transcript_id ""STRG.4.1""; re...",6.622187,ENSMUST00000182774.2
4,chr1,StringTie,transcript,4762442,4763647,1000,-,.,"gene_id ""STRG.5""; transcript_id ""STRG.5.1""; re...",0.312567,ENSMUST00000193443.2


In [8]:
%cd ../binfo1-own-analysis/
bed = pd.read_csv('gencode-exons-containing-startcodon.bed', sep='\t', header=None)
bed.columns = ['chromosome', 'start', 'end', 'transcript_id', 'start_codon', 'strand']
bed.head()

/qbio/mahoon2/bioinfo1/binfo1-own-analysis


,chromosome,start,end,transcript_id,start_codon,strand
0,chr1,3740774,3741721,ENSMUST00000070533.5,3741568,-
1,chr1,4422424,4423060,ENSMUST00000027032.6,4423045,-
2,chr1,4563322,4563689,ENSMUST00000027035.10,4563626,-
3,chr1,4563322,4563713,ENSMUST00000116652.8,4563626,-
4,chr1,4563322,4563958,ENSMUST00000191939.2,4563626,-


In [ ]:
merged_df = pd.merge(bed, df, on='transcript_id', how='left')
merged_df['TPM'] = merged_df['TPM'].fillna(0)
idx = merged_df.groupby(['chromosome_x', 'strand_x', 'start_codon'])['TPM'].idxmax()
principal_exons_df = merged_df.loc[idx]
principal_exons_df.head()

,chromosome_x,start_x,end_x,transcript_id,start_codon,strand_x,chromosome_y,program,type,start_y,end_y,unnamed1,strand_y,unnamed2,info,TPM
11,chr1,4878045,4878205,ENSMUST00000027036.11,4878136,+,chr1,StringTie,transcript,4878046.0,4916962.0,1000.0,+,.,"gene_id ""STRG.28""; transcript_id ""STRG.28.1""; ...",58.482693
13,chr1,4928036,4928199,ENSMUST00000081551.14,4928136,+,chr1,StringTie,transcript,4928037.0,4968128.0,1000.0,+,.,"gene_id ""STRG.29""; transcript_id ""STRG.29.1""; ...",128.363068
15,chr1,5154639,5154786,ENSMUST00000044369.13,5154673,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
16,chr1,5659227,5659528,ENSMUST00000027038.11,5659271,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
18,chr1,6300182,6300297,ENSMUST00000027040.13,6300226,+,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [14]:
bedout = principal_exons_df[['chromosome_x', 'start_x', 'end_x', 'transcript_id', 'start_codon', 'strand_x']]
bedout.columns = ['chromosome', 'start', 'end', 'transcript_id', 'start_codon', 'strand']
bedout.to_csv('gencode-principal-exons-containing-startcodon.bed', sep='\t', header=False, index=False)

In [ ]:
# Generate new bam file with only mapped reads
!(samtools view -H ../binfo1-datapack1/RPF-siLuc.bam; samtools view -F 4 ../binfo1-datapack1/RPF-siLuc.bam | bioawk -c sam '{ if (length($seq) >= 25) print $0; }') | samtools view -b -o  RPF-siLuc-filtered.bam
!(samtools view -H ../binfo1-datapack1/RPF-siLin28a.bam; samtools view -F 4 ../binfo1-datapack1/RPF-siLin28a.bam | bioawk -c sam '{ if (length($seq) >= 25) print $0; }') | samtools view -b -o  RPF-siLin28a-filtered.bam
!samtools index RPF-siLuc-filtered.bam
!samtools index RPF-siLin28a-filtered.bam

In [2]:
# Get 5' ends of +/- strand reads
!bedtools genomecov -ibam RPF-siLuc-filtered.bam -bg -5 -strand + > fivepcnt-RPF-siLuc.plus.bed
!bedtools genomecov -ibam RPF-siLuc-filtered.bam -bg -5 -strand - > fivepcnt-RPF-siLuc.minus.bed

In [3]:
!bedtools genomecov -ibam RPF-siLin28a-filtered.bam -bg -5 -strand + > fivepcnt-RPF-siLin28a.plus.bed
!bedtools genomecov -ibam RPF-siLin28a-filtered.bam -bg -5 -strand - > fivepcnt-RPF-siLin28a.minus.bed

In [16]:
!bedtools intersect -a fivepcnt-RPF-siLuc.plus.bed -b gencode-principal-exons-containing-startcodon.bed -wa -wb -nonamecheck > fivepcnt-RPF-siLuc.plus.txt
!bedtools intersect -a fivepcnt-RPF-siLuc.minus.bed -b gencode-principal-exons-containing-startcodon.bed -wa -wb -nonamecheck > fivepcnt-RPF-siLuc.minus.txt

In [15]:
!bedtools intersect -a fivepcnt-RPF-siLin28a.plus.bed -b gencode-principal-exons-containing-startcodon.bed -wa -wb -nonamecheck > fivepcnt-RPF-siLin28a.plus.txt
!bedtools intersect -a fivepcnt-RPF-siLin28a.minus.bed -b gencode-principal-exons-containing-startcodon.bed -wa -wb -nonamecheck > fivepcnt-RPF-siLin28a.minus.txt

In [ ]:
# TODO: determine P-site offset w.r.t. read length

import pysam
bamfile = pysam.AlignmentFile("RPF-siLuc-filtered.bam", "rb")

# 1. 개시 코돈 BED 파일을 한 줄씩 읽습니다.
for line in open("gencode-longest-exons-containing-startcodon.bed"):
    chrom, exon_start, exon_end, name, start_codon_pos, strand = line.strip().split('\t')

# 2. 해당 개시 코돈 주변의 Ribo-seq 리드를 BAM 파일에서 추출합니다.
    for read in bamfile.fetch(chrom, int(start_codon_pos) - 50, int(start_codon_pos) + 50):

# 3. 리드의 5' 말단 위치와 개시 코돈 위치 사이의 거리를 계산하여 저장합니다.
        read_5_end = read.reference_start if strand == '+' else read.reference_end
        distance = read_5_end - int(start_codon_pos)
        # 이 distance 값을 리드 길이(read.query_length)와 함께 저장
        print(f"{distance}\t{read.query_length}")